In [1]:
import pandas as pd
import numpy as np
import cProfile
import pstats
import io
import datetime
import math
import numba

from IPython.display import display, HTML

In [2]:
np.random.seed(17)

In [3]:
# !pip install watermark ipython-autotime
# !pip install line_profiler

In [4]:
%load_ext watermark
%load_ext autotime
%load_ext line_profiler

time: 56.9 ms (started: 2022-05-09 17:42:51 +01:00)


In [85]:
watermark --python -p pandas,numpy,numba

Python implementation: CPython
Python version       : 3.7.12
IPython version      : 7.33.0

pandas: 1.3.5
numpy : 1.19.5
numba : 0.55.1

time: 100 ms (started: 2022-05-10 08:26:28 +01:00)


## TOC:
* [Python, Pandas and Numpy](#ppp)
* [Slicing and assignment](#slice_and_assign)
* [Performance](#performance)
* [Misc](#misc)

#### Sources and extra readings:

A considerable source for this is the series written by Tom Augspurger (core pandas maintainer) on modern Pandas
https://tomaugspurger.github.io/modern-1-intro

Also, Ian Ozsvald has written a lot about high performance python (and about pandas) https://ianozsvald.com/ or https://github.com/ianozsvald/notes_to_self

And James Powell has published a few videos on pandas, eg https://www.youtube.com/watch?v=mWtfZaT7iSc

I would say that Auspurger and Powell are two super-users. So, if you want to see hardcore pandas use you may give a look at their code

# Python, Pandas and Numpy <a class="anchor" id="ppp"></a>

## Pandas is faster, because numpy is faster

In [6]:
def python_on_numpy_max(_vals):
    return max(_vals)
    
def numpy_max(_vals):
    return np.max(_vals)
    
def pandas_series_max(series):
    return series.max()

def python_on_series_max(_series):
    return max(_series)

def python_on_python_max(_python_list):
    return max(_python_list)

def save_profiling_results_to_file(pr, filename):
    result = io.StringIO()
    pstats.Stats(pr,stream=result).print_stats()
    result=result.getvalue()

    # chop the string into a csv-like buffer
    result='ncalls'+result.split('ncalls')[-1]
    result='\n'.join([','.join(line.rstrip().split(None,5)) for line in result.split('\n')])
    # save it to disk

    with open(filename, 'w+') as f:
        f.write(result)
        f.close()

time: 920 µs (started: 2022-05-09 17:42:51 +01:00)


In [7]:
vals = np.random.choice(range(100), size=100_000_000)

time: 2.43 s (started: 2022-05-09 17:42:51 +01:00)


In [8]:
pr = cProfile.Profile()
pr.enable()
python_on_numpy_max(vals)
pr.disable()

time: 9.29 s (started: 2022-05-09 17:42:54 +01:00)


In [9]:
save_profiling_results_to_file(pr, "profilings/python_on_numpy.csv")

time: 19.7 ms (started: 2022-05-09 17:43:03 +01:00)


In [10]:
pr = cProfile.Profile()
pr.enable()
numpy_max(vals)
pr.disable()

time: 135 ms (started: 2022-05-09 17:43:03 +01:00)


In [11]:
save_profiling_results_to_file(pr, "profilings/numpy.csv")

time: 28.1 ms (started: 2022-05-09 17:43:03 +01:00)


In [12]:
python_list = vals.tolist()

pr = cProfile.Profile()
pr.enable()
python_on_python_max(python_list)
pr.disable()

time: 3.4 s (started: 2022-05-09 17:43:03 +01:00)


In [13]:
save_profiling_results_to_file(pr, "profilings/python_on_python.csv")

time: 17.1 ms (started: 2022-05-09 17:43:07 +01:00)


In [14]:
series = pd.Series(vals)

pr = cProfile.Profile()
pr.enable()
pandas_series_max(series)
pr.disable()

time: 112 ms (started: 2022-05-09 17:43:07 +01:00)


In [15]:
save_profiling_results_to_file(pr, "profilings/pandas_max.csv")

time: 511 ms (started: 2022-05-09 17:43:07 +01:00)


## python (non vectorized) on pandas/numpy is slower than python on python

In [16]:
_ = python_on_series_max(series)

time: 9.63 s (started: 2022-05-09 17:43:07 +01:00)


In [17]:
_ = python_on_numpy_max(vals)

time: 9.48 s (started: 2022-05-09 17:43:17 +01:00)


In [18]:
_ = python_on_python_max(python_list)

time: 1.66 s (started: 2022-05-09 17:43:27 +01:00)


In [19]:
for name in ["python_on_numpy.csv",  "python_on_python.csv", "numpy.csv", "pandas_max.csv"]:
    df = pd.read_csv(f"profilings/{name}")
    df['filename:lineno(function)'] = df['filename:lineno(function)'].str.replace("/home/ubuntu/pyenvs/sandbox/lib/python3.7/site-packages/","")
    print(f"{name} df \n")
    if name in {"python_on_python.csv",  "python_on_numpy.csv"}:
        display(df)
    else:
        display(df[df["percall.1"]>0])
    print("\n\n\n")

python_on_numpy.csv df 



/home/ubuntu/pyenvs/sandbox/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


,ncalls,tottime,percall,cumtime,percall.1,filename:lineno(function)
0,1,0.000,0.000,9.294,9.294,/tmp/ipykernel_4183/3850208658.py:3(<module>)
1,1,0.000,0.000,0.000,0.000,/tmp/ipykernel_4183/3850208658.py:4(<module>)
2,1,0.000,0.000,9.294,9.294,/tmp/ipykernel_4183/1235816665.py:1(python_on_...
3,1,0.000,0.000,0.000,0.000,{method 'disable' of '_lsprof.Profiler' objects}
4,2,0.000,0.000,0.000,0.000,IPython/core/hooks.py:103(__call__)
5,2,0.000,0.000,0.000,0.000,IPython/core/hooks.py:168(pre_run_code_hook)
6,1,0.000,0.000,0.000,0.000,IPython/core/interactiveshell.py:124(<lambda>)
7,2,0.000,0.000,0.000,0.000,IPython/core/interactiveshell.py:1301(user_glo...
8,2,0.000,0.000,0.000,0.000,IPython/core/interactiveshell.py:3336(_update_...
9,2,0.000,0.000,0.000,0.000,IPython/core/interactiveshell.py:3452(compare)






python_on_python.csv df 



/home/ubuntu/pyenvs/sandbox/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


...

,ncalls,tottime,percall,cumtime,percall.1,filename:lineno(function)
0,1,0.000,0.000,0.135,0.135,/tmp/ipykernel_4183/2344467033.py:3(<module>)
2,1,0.000,0.000,0.135,0.135,/tmp/ipykernel_4183/1235816665.py:4(numpy_max)
3,1,0.000,0.000,0.135,0.135,<__array_function__ internals>:2(amax)
5,1,0.000,0.000,0.135,0.135,numpy/core/fromnumeric.py:70(_wrapreduction)
7,1,0.000,0.000,0.135,0.135,numpy/core/fromnumeric.py:2589(amax)
8,1,0.000,0.000,0.135,0.135,{built-in method numpy.core._multiarray_umath....
9,1,0.135,0.135,0.135,0.135,{method 'reduce' of 'numpy.ufunc' objects}
17,2,0.000,0.000,0.135,0.067,IPython/core/interactiveshell.py:3511(run_code)
29,2,0.000,0.000,0.135,0.067,{built-in method builtins.exec}






pandas_max.csv df 



/home/ubuntu/pyenvs/sandbox/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


,ncalls,tottime,percall,cumtime,percall.1,filename:lineno(function)
0,1,0.00,0.000,0.111,0.111,/tmp/ipykernel_4183/1727385726.py:5(<module>)
2,1,0.00,0.000,0.111,0.111,/tmp/ipykernel_4183/1235816665.py:7(pandas_ser...
4,1,0.00,0.000,0.111,0.111,pandas/core/series.py:4359(_reduce)
7,1,0.00,0.000,0.111,0.111,pandas/core/generic.py:10324(_stat_function)
8,1,0.00,0.000,0.111,0.111,pandas/core/generic.py:10363(max)
9,1,0.00,0.000,0.111,0.111,pandas/core/generic.py:10806(max)
11,1,0.00,0.000,0.111,0.111,pandas/core/nanops.py:120(f)
16,1,0.00,0.000,0.111,0.111,pandas/core/nanops.py:396(new_func)
17,1,0.00,0.000,0.110,0.110,pandas/core/nanops.py:997(reduction)
40,1,0.00,0.000,0.110,0.110,numpy/core/_methods.py:37(_amax)






time: 86.1 ms (started: 2022-05-09 17:43:28 +01:00)


## Conclusions
- Often pandas is just a wrapper around numpy. The more you move away from doing stuff that is naturally in numpy, ie vectorized operations, the less performant your code will be
- numpy and pandas are performant if and only if you stick to vectorized numpy and pandas operations. They are not more performant by definition

## Why is pandas faster? (aka why is numpy faster?)
It is a wrapper around numpy, which is faster

Numpy is faster because of a few different reasons:

1. In numpy data is stored in a contiguous C array. So, the data points are stored one after the other, which makes access faster (Locality of reference https://en.wikipedia.org/wiki/Locality_of_reference). In python you store pointers to objects in a list.

In [20]:
foo = np.array([1, 2, 3])

time: 620 µs (started: 2022-05-09 17:43:28 +01:00)


In [21]:
id(foo[0])

139844586109008

time: 3.01 ms (started: 2022-05-09 17:43:28 +01:00)


In [22]:
id(foo[0])

139844586108080

time: 1.98 ms (started: 2022-05-09 17:43:28 +01:00)


In [23]:
# the object is created on the fly when you access it through an index
foo[0] is foo[0]

False

time: 2.18 ms (started: 2022-05-09 17:43:28 +01:00)


In [24]:
bar = [1, 2, 3]

time: 7.07 ms (started: 2022-05-09 17:43:28 +01:00)


In [25]:
id(bar[0])

139846703192576

time: 2.5 ms (started: 2022-05-09 17:43:28 +01:00)


In [26]:
id(bar[0])

139846703192576

time: 2.14 ms (started: 2022-05-09 17:43:28 +01:00)


In [27]:
# the same object is accessed through the pointer stored in the list
bar[0] is bar[0]

True

time: 3.57 ms (started: 2022-05-09 17:43:28 +01:00)


2. Also, by definition they are also stored with single type. While in python when you stored integers in a list you are storing pointers to objects. The objects can be whatever, and the type has to be checked before operations can be ran. This is also the reason why numpy/pandas are more memory efficient

In [28]:
foo = np.array([1, 2, 3])

time: 483 µs (started: 2022-05-09 17:43:29 +01:00)


In [29]:
foo[2] = "a"

ValueError: invalid literal for int() with base 10: 'a'

time: 216 ms (started: 2022-05-09 17:43:29 +01:00)


In [30]:
bar = list([1, "a", 3.5])

time: 509 µs (started: 2022-05-09 17:48:06 +01:00)


In [31]:
bar

[1, 'a', 3.5]

time: 3.18 ms (started: 2022-05-09 17:48:06 +01:00)


3. Many numpy operations are implement in C and operates directly on the array, rather than having to do expensive loops like python does. The first is faster. Also, even if the operation has been implemented in cPython it still needs to move the data from python objects to C consumable objects, which has some overhead. Compared to numpy where the arrays can already consumed by C

python max implementation is in Cpython but still loops through the iterable provided as argument
https://github.com/python/cpython/blob/da20d7401de97b425897d3069f71f77b039eb16f/Python/bltinmodule.c#L1756

## Sources (use the stackoverflow answers as a starting point)
- https://stackoverflow.com/questions/40911491/some-confusions-on-how-numpy-array-stored-in-python
- https://stackoverflow.com/questions/8385602/why-are-numpy-arrays-so-fast
- https://www.reddit.com/r/Python/comments/jorq7/how_numpy_is_faster_in_computation/


# Slicing and assignments <a class="anchor" id="slice_and_assign"></a>

## Using indexes may not feel natural at first but getting used to them will simplify many operations

In [32]:
size =1_000
df = pd.DataFrame({
    "cat1": np.random.choice(["a", "b", "c"], size=size, p=[0.6, 0.3, 0.1]),
    "cat2": np.random.choice(["d", "e", "f"], size=size, p=[0.6, 0.3, 0.1]),
    "numbers": np.random.choice(range(10), size=size),
}).set_index(["cat1", "cat2"]).sort_index()

time: 6.78 ms (started: 2022-05-09 17:48:07 +01:00)


## Indexes make slicing way less verbose

In [33]:
# this is way less verbose than ...
df.loc[("a", "f"),"numbers"].head()

cat1  cat2
a     f       1
      f       4
      f       1
      f       1
      f       9
Name: numbers, dtype: int64

time: 5.96 ms (started: 2022-05-09 17:48:07 +01:00)


In [34]:
# this
df_temp = df.reset_index()
df_temp[(df_temp['cat1'] == "a") & (df_temp['cat2']=="f")]["numbers"].head()

525    1
526    4
527    1
528    1
529    9
Name: numbers, dtype: int64

time: 10.2 ms (started: 2022-05-09 17:48:07 +01:00)


## Indexes make alignment automatic for most operations

In [35]:
df_copy = df.copy().assign(numbers=df["numbers"].clip(2,))

time: 2.58 ms (started: 2022-05-09 17:48:07 +01:00)


In [36]:
stats = df_copy.groupby(level=[0,1])["numbers"].agg(["min", "max"])
df_copy.assign(
    numbers_minmaxed=((df_copy['numbers']-stats['min'])/(stats['max']-stats['min'])).round(2)
).tail(10)

numbers  numbers_minmaxed
cat1 cat2                           
c    e           7              0.71
     e           3              0.14
     e           5              0.43
     e           2              0.00
     e           3              0.14
     e           7              0.71
     f           2              0.00
     f           6              0.80
     f           2              0.00
     f           7              1.00

time: 23 ms (started: 2022-05-09 17:48:07 +01:00)


In [37]:
# an extra merge and an extra drop
stats_no_idx = df_copy.groupby(level=[0,1])["numbers"].agg(["min", "max"]).reset_index()
(
    df_copy.reset_index()
     .merge(stats_no_idx, on=['cat1', 'cat2'], how="left")
     .assign(numbers_minmaxed=lambda x: ((x['numbers']-x['min'])/(x['max']-x['min'])).round(2))
     .drop(["min", "max"], axis=1)
).tail(10)

,cat1,cat2,numbers,numbers_minmaxed
990,c,e,7,0.71
991,c,e,3,0.14
992,c,e,5,0.43
993,c,e,2,0.00
994,c,e,3,0.14
995,c,e,7,0.71
996,c,f,2,0.00
997,c,f,6,0.80
998,c,f,2,0.00
999,c,f,7,1.00


time: 26.2 ms (started: 2022-05-09 17:48:07 +01:00)


## Fifty shades of slicing

In [38]:
# use index values
df.loc[("a", "f"),"numbers"]

cat1  cat2
a     f       1
      f       4
      f       1
      f       1
      f       9
             ..
      f       1
      f       3
      f       2
      f       7
      f       0
Name: numbers, Length: 70, dtype: int64

time: 7.71 ms (started: 2022-05-09 17:48:07 +01:00)


In [39]:
# use index position
df.iloc[np.argwhere(df.index==('a','f')).ravel()]["numbers"]

cat1  cat2
a     f       1
      f       4
      f       1
      f       1
      f       9
             ..
      f       1
      f       3
      f       2
      f       7
      f       0
Name: numbers, Length: 70, dtype: int64

time: 7.04 ms (started: 2022-05-09 17:48:07 +01:00)


In [40]:
pd.IndexSlice[:,["f", "e"]]

(slice(None, None, None), ['f', 'e'])

time: 2.28 ms (started: 2022-05-09 17:48:08 +01:00)


In [41]:
df.loc[pd.IndexSlice[:,["f", "e"]], "numbers"]
# a cleaner version of df.loc[(slice(None, None, None), ["f", "e"]), "numbers"]
# https://stackoverflow.com/questions/44087637/pandas-how-does-indexslice-work

cat1  cat2
a     f       1
      f       4
      f       1
      f       1
      f       9
             ..
c     e       3
      e       5
      e       2
      e       3
      e       7
Name: numbers, Length: 402, dtype: int64

time: 17.8 ms (started: 2022-05-09 17:48:08 +01:00)


In [42]:
df.loc[(df.index==('a','f')) & (df['numbers']==3), "numbers"]

cat1  cat2
a     f       3
      f       3
      f       3
      f       3
      f       3
      f       3
Name: numbers, dtype: int64

time: 5.36 ms (started: 2022-05-09 17:48:08 +01:00)


## Assignements

In [43]:
df_copy = df.copy()

time: 620 µs (started: 2022-05-09 17:48:08 +01:00)


In [44]:
df['numbers'].value_counts().sort_index()

0     85
1     97
2    109
3    124
4    122
5     95
6     84
7     95
8     79
9    110
Name: numbers, dtype: int64

time: 5.42 ms (started: 2022-05-09 17:48:08 +01:00)


In [45]:
df.loc[(df.index==('a','f')) & (df['numbers']==3), "numbers"] = 3 ** 4

time: 2.22 ms (started: 2022-05-09 17:48:08 +01:00)


In [46]:
# assign it back
df.loc[(df.index==('a','f')) & (df['numbers']==81), "numbers"] = 3

time: 1.87 ms (started: 2022-05-09 17:48:08 +01:00)


In [47]:
df['numbers'] = np.where((df.index==('a','f')) & (df['numbers']==3), 3 **4, df['numbers'])

time: 1.9 ms (started: 2022-05-09 17:48:08 +01:00)


In [48]:
df_copy['numbers'] = np.where(
    (df_copy.index==('a','f')) & (df_copy['numbers']==3), 3 **4, 
    np.where((df_copy.index==('a','f')) & (df_copy['numbers']==4), 4 ** 5,
             np.where((df_copy.index==('a','f')) & (df_copy['numbers']==5), 5 ** 6,
             df_copy['numbers'])))

time: 612 ms (started: 2022-05-09 17:48:08 +01:00)


In [49]:
df_copy['numbers'].value_counts().sort_index()

0         85
1         97
2        109
3        118
4        115
5         93
6         84
7         95
8         79
9        110
81         6
1024       7
15625      2
Name: numbers, dtype: int64

time: 3.6 ms (started: 2022-05-09 17:48:08 +01:00)


In [50]:
df_copy = df.copy()

time: 607 µs (started: 2022-05-09 17:48:08 +01:00)


In [51]:
## The above is arguably hard to read, so we can achieve the same result with something like. I think that this is the nicest and most flexible
conditions, choices = zip(*[
    # ((condition), choice)
    ((df_copy.index==('a','f')) & (df_copy['numbers']==3), 3 **4),
    ((df_copy.index==('a','f')) & (df_copy['numbers']==4), 4 ** 5),
    ((df_copy.index==('a','f')) & (df_copy['numbers']==5), 5 ** 6),
])
df_copy['numbers'] = pd.Series(np.select(conditions, choices, default=np.nan), index=df_copy.index).fillna(df_copy['numbers'])

time: 5.74 ms (started: 2022-05-09 17:48:08 +01:00)


In [52]:
df_copy['numbers'].value_counts().sort_index()

0.0         85
1.0         97
2.0        109
3.0        118
4.0        115
5.0         93
6.0         84
7.0         95
8.0         79
9.0        110
81.0         6
1024.0       7
15625.0      2
Name: numbers, dtype: int64

time: 5.96 ms (started: 2022-05-09 17:48:08 +01:00)


In [53]:
df_copy = df.copy()

time: 754 µs (started: 2022-05-09 17:48:09 +01:00)


In [54]:
# Or given that we are always using the same indexes and only changing behavior according to value, we can also use a mapper
mapper = {3: 4, 4:5, 5:6}
df_copy.loc[("a", "f"), "numbers"] = (
    df_copy.loc[("a", "f")]["numbers"] ** df_copy.loc[("a", "f")]["numbers"].map(mapper).fillna(1)
)

time: 6.46 ms (started: 2022-05-09 17:48:09 +01:00)


In [55]:
df_copy['numbers'].value_counts().sort_index()

0.0         85
1.0         97
2.0        109
3.0        118
4.0        115
5.0         93
6.0         84
7.0         95
8.0         79
9.0        110
81.0         6
1024.0       7
15625.0      2
Name: numbers, dtype: int64

time: 4.45 ms (started: 2022-05-09 17:48:09 +01:00)


# Performance <a class="anchor" id="performance"></a>

## Also, when combining multiple dataframes concatenate them rather than append each one of the sequentially to the main one

```
Depending on the values, pandas might have to recast the data to a different type. And indexes are immutable, so each time you append pandas has to create an entirely new one.
```
https://tomaugspurger.github.io/modern-4-performance

## Vectorize, vectorize, vectorize

For what concerns writing efficient pandas getting used to indexes goes hand in hand with getting used to think in terms of vectorized operations. You should always try to vectorize your operation, avoid at all cost `.apply` and/or iterations. There are many ways to do so:

1. use in-built pandas type specific methods, ie series.max(), series.mean(), series.str.whatever (these include regex ones and slicing `.str[:5]`)

In [56]:
size =1_000_000
df = pd.DataFrame({
    "cat1": np.random.choice(["a", "b", "c"], size=size, p=[0.6, 0.3, 0.1]),
    "cat2": np.random.choice(["d", "e", "f"], size=size, p=[0.6, 0.3, 0.1]),
    "numbers": np.random.choice(range(10), size=size),
}).set_index(["cat1", "cat2"]).sort_index()

time: 506 ms (started: 2022-05-09 17:48:09 +01:00)


In [57]:
# when I started working with pandas this felt much more natural to me

# again min_max scaling within groups
df_copy = df.copy()
df_copy.groupby(level=[0,1])["numbers"].apply(lambda x: (x-x.min())/(x.max()-x.min()))

cat1  cat2
a     d       0.000000
      d       0.888889
      d       0.666667
      d       0.666667
      d       0.333333
                ...   
c     f       0.666667
      f       0.777778
      f       0.222222
      f       0.000000
      f       0.888889
Name: numbers, Length: 1000000, dtype: float64

time: 1.02 s (started: 2022-05-09 17:48:09 +01:00)


In [58]:
# but if vectorized leads to a significant improvement in performance
grouped = df_copy.groupby(level=[0,1])["numbers"]
(df_copy.numbers - grouped.transform(np.min))/(grouped.transform(np.max)-grouped.transform(np.min))

cat1  cat2
a     d       0.000000
      d       0.888889
      d       0.666667
      d       0.666667
      d       0.333333
                ...   
c     f       0.666667
      f       0.777778
      f       0.222222
      f       0.000000
      f       0.888889
Name: numbers, Length: 1000000, dtype: float64

time: 109 ms (started: 2022-05-09 17:48:10 +01:00)


## If you cannot vectorize with pandas

2. use numpy operations
3. use Numba (JIT compiler that translates a subset of Python and NumPy code into fast machine code)
4. use Cython (gives C-like performance with code that is written mostly in python with C-inspired syntax)
5. possibly use `eval`. I have never tried it, so cannot comment on that https://pandas.pydata.org/docs/user_guide/enhancingperf.html#expression-evaluation-via-eval
6. parallelize through Dask

### The difference in speed between the two cells below is something to think about. Numpy is likely to be faster than pandas
more details here https://speakerdeck.com/ianozsvald/making-pandas-fly-europython-2020?slide=15

In [82]:
df_copy['numbers'].sum()

4500090

time: 4.03 ms (started: 2022-05-09 18:01:41 +01:00)


In [83]:
df_copy['numbers'].values.sum()

4500090

time: 2.56 ms (started: 2022-05-09 18:01:48 +01:00)


### Vectorize with numpy or numba

In [60]:
def dummy_op(value):
    return math.log(value+1)

time: 758 µs (started: 2022-05-09 17:48:13 +01:00)


In [61]:
bar_apply = df_copy["numbers"].apply(dummy_op)

time: 491 ms (started: 2022-05-09 17:48:13 +01:00)


In [62]:
bar_numpy = np.log(df_copy["numbers"]+1)

time: 30.9 ms (started: 2022-05-09 17:48:13 +01:00)


In [63]:
@numba.jit
def dummy_op_numba(array):
    n = len(array)
    result = np.empty(n, dtype="float64")
    for i in range(n):
        result[i] = math.log(array[i]+1)
    return result

time: 51.5 ms (started: 2022-05-09 17:48:13 +01:00)


In [64]:
bar_numba = dummy_op_numba(df_copy['numbers'].to_numpy())

time: 3.05 s (started: 2022-05-09 17:48:13 +01:00)


In [65]:
assert all(pd.Series(bar_apply.to_numpy()) == pd.Series(bar_numpy.to_numpy()))
assert all(pd.Series(bar_apply.to_numpy()) == pd.Series(bar_numba))

time: 244 ms (started: 2022-05-09 17:48:16 +01:00)


## The right data type
Also, often times vectorized operations are only available when using the right data type (eg `.str`, `.dt` etc)

```
pandas operations are optimized for data types, so most of the time you want to use data types and their related operations
```
a famous quote from Captain obvious

which is almost always the right way....

In [66]:
dates = 100
base = datetime.datetime.today()
date_list = [(base - datetime.timedelta(days=x)).date().isoformat() for x in range(dates)]

time: 1.08 ms (started: 2022-05-09 17:48:17 +01:00)


In [67]:
df = pd.DataFrame({"date":np.tile(date_list, int(20_000_000/50))})
df.dtypes

date    object
dtype: object

time: 9.42 s (started: 2022-05-09 17:48:17 +01:00)


In [68]:
df['date_good_type'] = pd.to_datetime(df['date'])
df['date_good_type_weekday'] = df['date_good_type'].dt.weekday

time: 16.3 s (started: 2022-05-09 17:48:26 +01:00)


In [69]:
mapper = {date_val: date_val.weekday() for date_val in pd.to_datetime(date_list)}
df['date_weekday'] = pd.Categorical(df['date']).map(mapper)

time: 6.52 s (started: 2022-05-09 17:48:42 +01:00)


In [70]:
assert all(df['date_weekday']==df['date_good_type_weekday'])

time: 5.11 s (started: 2022-05-09 17:48:49 +01:00)


## Combining dataframes

I assume you are all familiar on how to combine dataframes using `pd.concat`, `pd.merge` and `pd.join`. If not it should be easy to find comparisons of these methods.
- https://pandas.pydata.org/docs/user_guide/merging.html
- https://stackoverflow.com/questions/38256104/differences-between-merge-and-concat-in-pandas

Something more exotic is running SQL-like left-excluding, right-excluding, and anti-joins

https://stackoverflow.com/questions/53645882/pandas-merging-101

In [71]:
np.random.seed(17)
left = pd.DataFrame({'key': ['A', 'B', 'C', 'D'], 'value': np.random.randn(4)})
right = pd.DataFrame({'key': ['B', 'D', 'E', 'F'], 'value': np.random.randn(4)})

time: 2.47 ms (started: 2022-05-09 17:48:54 +01:00)


In [72]:
left

,key,value
0,A,0.276266
1,B,-1.854628
2,C,0.623901
3,D,1.145311


time: 8.04 ms (started: 2022-05-09 17:48:54 +01:00)


In [73]:
right

,key,value
0,B,1.037190
1,D,1.886639
2,E,-0.111698
3,F,-0.362101


time: 8.76 ms (started: 2022-05-09 17:48:54 +01:00)


In [74]:
left.merge(right, on='key', how='left', indicator=True).query('_merge == "left_only"').drop('_merge', 1)

/home/ubuntu/pyenvs/sandbox/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


,key,value_x,value_y
0,A,0.276266,NaN
2,C,0.623901,NaN


time: 43.4 ms (started: 2022-05-09 17:48:54 +01:00)


In [75]:
left.merge(right, on='key', how='left', indicator=True)

,key,value_x,value_y,_merge
0,A,0.276266,NaN,left_only
1,B,-1.854628,1.037190,both
2,C,0.623901,NaN,left_only
3,D,1.145311,1.886639,both


time: 28.3 ms (started: 2022-05-09 17:48:54 +01:00)


In [76]:
left.merge(right, on='key', how='right', indicator=True).query('_merge == "right_only"').drop("_merge", 1)

/home/ubuntu/pyenvs/sandbox/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


,key,value_x,value_y
2,E,NaN,-0.111698
3,F,NaN,-0.362101


time: 37.2 ms (started: 2022-05-09 17:48:54 +01:00)


In [77]:
left.merge(right, on='key', how='outer', indicator=True).query('_merge != "both"').drop("_merge", 1)

/home/ubuntu/pyenvs/sandbox/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


,key,value_x,value_y
0,A,0.276266,NaN
2,C,0.623901,NaN
4,E,NaN,-0.111698
5,F,NaN,-0.362101


time: 48.6 ms (started: 2022-05-09 17:48:54 +01:00)


# Misc  <a class="anchor" id="misc"></a>

## Consider whether you want to use first

In [78]:
df = pd.DataFrame({'id' : [1,1,1,2,2,3,3,3,3,4,4],
            'value'  : ["first","second","third", np.NaN,
                        "second","first","second","third",
                        "fourth","first","second"]})


time: 1.27 s (started: 2022-05-09 17:48:54 +01:00)


In [79]:
 df.groupby('id').nth(0)

,value
id,
1,first
2,NaN
3,first
4,first


time: 13.1 ms (started: 2022-05-09 17:48:56 +01:00)


In [80]:
 df.groupby('id').first()

,value
id,
1,first
2,second
3,first
4,first


time: 13.6 ms (started: 2022-05-09 17:48:56 +01:00)


## Do not use inplace=True

it is not necessarily more performant, and it does not necessairly make you save memory (it may still need to create a copy of the dataframe)

```
There is no guarantee that an inplace operation is actually faster. Often they are actually the same operation that works on a copy, but the top-level reference is reassigned.

[...]

My personal opinion: I never use in-place operations. The syntax is harder to read and it does not offer any advantages.
```
[Jeff Reback (pandas core developer)](https://pandas.pydata.org/about/team.html)

https://stackoverflow.com/questions/22532302/pandas-peculiar-performance-drop-for-inplace-rename-after-dropna/22533110#22533110

Also, it will be deprecated with pandas 2.0

https://github.com/pandas-dev/pandas/issues/16529

## Other very useful methods

- `shift` (and `ffill`) often this is a very handy method for coming up with a vectorized way of doing something which naturally feels like requiring iterations. eg detect step changes in a time series, rather than iterating and checking the current value vs the previou one you can shift forward by whatever number of steps necessary and then compare values on the same row
- `stack`/`melt` and `unstack` turns wide data too long and viceversa
- It may be worth skimming the API page, in case you haven't done it yet https://pandas.pydata.org/docs/reference/frame.html